# Wasserstein GAN
> Martin Arjovsky(Courant Institute of Mathematical Sciences), Soumith Chintala(Facebook AI Research), and L´eon Bottou1(Courant Institute of Mathematical Sciences, Facebook AI Research)

- toc:true
- branch: master
- badges: false
- comments: false 
- author: 최서연
- categories: [Wasserstein GAN, GAN]

ref: https://arxiv.org/pdf/1701.07875.pdf

https://ahjeong.tistory.com/7

wgan math: https://www.slideshare.net/ssuser7e10e4/wasserstein-gan-i

## Different Distances

- Let $X$ be a compact metric set (such as the space of images $[0, 1]^d$) 
    - compact하다는 것은 close하고 bounded 하다는 것!
- and let $Σ$ denote the set of all the Borel subsets of $X$ . 
- Let $Prob(X )$ denote the space of probability measures defined on $X$ .
- We can now define elementary distances and divergences between two distributions $P_r, P_g ∈ Prob(X )$
    - The Total Variation (TV) distance
        - Total Variation(TV)은 두 확률측도의 측정값이 벌어질 수 있는 값 중 가장 큰 값(혹은  supremum)
        - 같은 집합 A라 하더라도 두 확률분포가 측정하는 값은 다를 수 있고, 이 때 TV는 모든 $A \in \Sigma$에 대해 가장 큰 값으로 정의한 것
        - 만약 두 확률분포의 확률밀도함수가 서로 겹치지 않는다면(확률분포의 suppot의 교집합이 공집합이라면) TV는 무조건 1
    - The Kullback-Leibler (KL) divergence
        - KL은 대칭성과 삼각부등식이 깨지기 때문에 metric이 아니지만, Premetric!
        - TV 보다 Strong함(즉, KL으로 TV 설명 가능), 그래서 TV에서 수렴하지 않으면 KL에서 수렴하지 않음
    - The Jensen-Shannon (JS) divergence
        - JS는 TV랑 equivalent 함.(즉, 서로를 표현 가능), 그래서 TV에서 수렴하지 않으면 JS에서 수렴하지 않음
    - The Earth-Mover (EM) distance or Wasserstein-1
    - p값들은 절대적으로 연속!
    - 그러므로 카이제곱에서 정의되는 같은 측정치 뮤가 나옴.
- The following example illustrates how apparently simple sequences of probability distributions converge under the EM distance but do not converge under the other distances and divergences defined above
    - 다음 예시에서 보기에 단순한 확률 분포 시퀀스가 EM 거리에서는 수렴하지만 제시된 4개의 거리 및 발산에서는 수렴하지 않음을 보임.

$X$: compact metric set
- Metric은 Distance라고도 불리며, 5가지 성분 만족하는 거리함수 d가 존재하는 걸 말함
    - $d(x,y) \geq 0$
    - $d(x,y) = 0$ 이면 $x=y$ 다.
    - $x=y$이면 $d(x,y) = 0$이다.
    - $d(x,y)=d(y,x)$: 대칭이다.
    - $d(x,y) \leq d(x,z) + d(z,y)$ 삼각부등식이 성립한다.
- metric의 개념의 중요성
    - 수렴convergence의 개념을 정의내릴 수 있기 때문.

*Example 1 (Learning parallel lines)*

- Let $Z ∼ U[0, 1]$ the uniform distribution on the unit interval.
- Let $P_0$ be the distribution of $(0, Z) ∈ R^2$ (a 0 on the x-axis and the random variable Z on the y-axis), uniform on a straight vertical line passing through the origin.
- Now let $g_θ(z) = (θ, z)$ with $θ$ a single real parameter. 
- It is easy to see that in this case,

- $\theta \neq 0$이라면 $P_0$와 $P_{\theta}$는 서로 겹치지 않는 확률분포이므로 TV가 1이 된다.
- $\theta \neq 0$이라면
    - $P_0 (x) \neq 0 \rightarrow P_{\theta}(x) = 0$
    - $P_{\theta} (x) \neq 0 \rightarrow P_{0}(x) = 0$
        - 따라서 $P_{\theta}>0$인 곳에서 $log\big( \frac{P_{\theta}(x)}{P_0 (x)} \big)$값은 무한이 된다.
        - 그래서 KL도 무한, 단, $\theta = 0$이 되지 않을 때
- $\theta \neq 0$이라면
    - $P_0 (x) \neq 0 \rightarrow P_{\theta}(x) = 0 \rightarrow P_m = \frac{P_0}{2} \rightarrow KL(P_0\|P_m) = log2$
    - $P_{\theta} (x) \neq 0 \rightarrow P_{0}(x) = 0 \rightarrow P_m = frac{P_{\theta}}{2} \rightarrow KL()P_{\theta}\|P_m) = log2$
        - JS는 KL처럼 무한대는 되지 않지만 TV처럼 $\theta \neq 0$일땐 log2로 일정한 값만 가지게 된다.
        - $\theta = 0$이면 0
**TV나 KL이나 JS는 두 확률분포 $\mathbb{P}_R, \mathbb{P}_G$가 서로 다른 영역에서 측정된 경우 완전히 다르다고 판단을 내리게끔 metric이 계산되기 때문, 즉, 두 확률 분포의 차이를 harsh하게 봄**
    - GAN에서는 discriminator의 학습이 잘 죽는 이유..
    - 따라서 GAN 학습의 유연성을 위해 수렴에 focus를 맞춘 metric 필요

- When $θ_t → 0$, the sequence ($P_{θ_t})_{t∈N}$ converges to $P_0$ under the EM distance, but does not converge at all under either the JS, KL, reverse KL, or TV divergences.
- Figure 1 illustrates this for the case of the EM and JS distances

- Example 1 gives us a case where we can learn a probability distribution over a low dimensional manifold by doing gradient descent on the EM distance. 
- This cannot be done with the other distances and divergences because the resulting loss function is not even continuous.
    - 결과 손실 함수가 연속적이지 않기 때문에 다른 거리와 발산으로 할 수 없음.
- Although this simple example features distributions with disjoint supports, the same conclusion holds when the supports have a non empty intersection contained in a set of measure zero.
    - disjoint suppoert를 가진 분포를 특장으로 하는 이 단순한 예시는 supports가 0 집합에 포함된 비어 있지 않은 interaction을 가진 때에도 동일한 결과가 유지된다.
- This happens to be the case when two low dimensional manifolds intersect in general position.

- 두 확률변수 $(X,Y)$가 각각 $X~\mathbb{P_0}, Y~\mathbb{P_{\theta}}$라고 할때, 각 $w$에 대해서 $X,Y$는 2치원 공간으로 매핑됨
    - $X(w) = (0,Z_1(w)), Y(w) = (\theta, Z_2(w))$
- 두 점 사이의 거리는 $d(X,Y) = (|\theta - 0|^2 + |Z_1(w) - Z_2(w)|)^{1/2} \geq |\theta|$로 계산
    - $d(X,Y)$의 기대값은 어떤 결함확률분포 $\gamma$ 를 사용하든 항상 $|\theta|$보다 크거나 같음.
    - $\mathbb{e}^{\gamma}[d(X,Y)] \geq \mathbb{E}^{\gamma} [|\theta|] = |\theta|$
        - $Z_1 = Z_2$인 분포를 따를떄 $d(X,Y) = (|\theta - 0|^2 + |Z_1(w) - Z_2(w)|)^{1/2}  = |\theta|$ 가능!
        - 따라서 $W(\mathbb{P_0}, \mathbb{P_{\theta}}) = |\theta|$

![](https://xiucheng.org/assets/images/wgan-example.png)

Since the Wasserstein distance is much weaker than the JS distance , we can now ask whether $W(P_r, P_θ)$ is a continuous loss function on $θ$ under mild assumptions. This, and more, is true, as we now state and prove.

*Theorem 1.*

- Let $P_r$ be a fixed distribution over $X$ .
- Let $Z$ be a random variable(e.g Gaussian) over another space $Z$. 
- Let $g : Z × R^d → X$ be a function, that will be denoted $g_θ(z)$ with $z$ the first coordinate and $θ$ the second.
- Let $P_θ$ denote the distribution of $g_θ(Z)$. Then,
1. If $g$ is continuous in $θ$, so is $W(P_r, P_θ)$.
    - g가 세타에 대해 연속이면 EM 거리도 연속!
2. If $g$ is locally Lipschitz and satisfies regularity assumption 1, then $W(P_r, P_θ)$ is continuous everywhere, and differentiable almost everywhere.
    - g가 립시츠 조건 만족한다면  EM 거리도 연속!
3. Statements 1-2 are false for the Jensen-Shannon divergence $JS(P_r, P_θ)$ and all the KLs.

The following corollary tells us that learning by minimizing the EM distance makes sense (at least in theory) with neural networks.
- 다음 결과는 EM 거리를 최소화하여 학습하는 것이 신경망에 타당하다는 것을 말해줌

*Corollary 1*

- Let $g_θ$ be any feedforward neural network parameterized by $θ$, and $p(z)$ a prior over $z$ such that $E_{z∼p(z)}(||z||) < ∞$ (e.g. Gaussian, uniform, etc.)
- Then assumption 1 is satisfied and therefore $W(P_r, P_θ)$ is continuous everywhere and differentiable almost everywhere.

All this shows that EM is a much more sensible cost function for our problem than at least the Jensen-Shannon divergence. The following theorem describes the relative strength of the topologies induced by these distances and divergences, with KL the strongest, followed by JS and TV, and EM the weakest.
- EM이 최소 JS 발산보다 제시한 문제에 보다 더 합리적인 cost function이라는 것을 보임.
- 상대적인 강도는 KL > JS > TV > EM 의 순을 이룸

*Theorem 2.*

- Let $P$ be a distribution on a compact space $X$ and $(P_n)_{n∈N}$ be a sequence of distributions on $X$ .
- Then, considering all limits as $n → ∞$, 
1. The following statements are equivalent 
    - $δ(P_n, P) → 0$ with δ the total variation distance 
    - $JS(P_n, P) → 0$ with JS the Jensen-Shannon divergence.
2. The following statements are equivalent
    - $W(P_n, P) → 0.$
    - $P_n D→ P$ where $D→$ represents convergence in distribution for random variables.
3. KL($P_n|P) → 0$ or KL($P|P_n) → 0$ imply the statements in (1).
4. The statements in (1) imply the statements in (2).

This highlights the fact that the KL, JS, and TV distances are not sensible cost functions when learning distributions supported by low dimensional manifolds. However the EM distance is sensible in that setup. This obviously leads us to the next section where we introduce a practical approximation of optimizing the EM distance.

- TV, KL, JS 는 $(\mathbb{P}_{r}, \mathbb{P}_g)$가 서로 겹치지 않는 상황에서는 불연속이 된다.
- EM(Wasserstein distance)은 TV, KL, JS 보다 약한weak metric이므로 수렴하는데 무른soft 성질을 가진다.
- EM은 분포수렴과 동등하다.
    - 분포수렴
        - 확률분포 수렴 종류 중 하나로서, 가장 약한weak 수렴.
        - 확률분포의 개별적인 특징보다 전체적인 모양을 중시하는 수렴
        - 중심극한정리에서 표본평균이 정규분포로 수렴하는 종류가 분포수렴
        - $X_n$의 모든 모멘트가 $X$의 모멘트로 수렴하면 분포수렴
        - $X_n$의 누적확률밀도함수가 $X$의 누적확률밀도함수 중 연속인 모든 점에서 수렴하면 분포수렴
        - $X_n$의 푸리에 변환Fourier transform이 수렴하면 분포수렴

## Wasserstein GAN

- Again, Theorem 2 points to the fact that $W(P_r, P_θ)$ might have nicer properties when optimized than $JS(P_r, P_θ)$. 
- However, the infimum in (1) is highly intractable. 
    - inf A = max{lower bound of A}, 즉 infmnum 은 the greatest lower bound로서, 하한값lower bound 중 가장 큰 값maximum임!
- On the other hand, the Kantorovich-Rubinstein duality [22] tells us that (2)
    - supA = min{upper bound of A}, 즉 Supermum은 the least upper bound 로서, 상한값upper bound 에서 가장 작은 값minimum임!


- WGAN, $W{\mathbb{P}, \mathbb{Q}} = \inf_{\substack{ \gamma \in \prod (\mathbb{P},\mathbb{Q}) }} \mathbb{E}^{\gamma} [d(X,Y)]$
    - $\prod (\mathbb{P}, \mathbb{Q})$는 두 확률분포 $\mathbb{P}, \mathbb{Q}$의 결합확률분포들을 모은 집합, $\gamma$는 그 중 하나.
    - 즉 모든 결합확률분포 $\prod (\mathbb{P}, \mathbb{Q})$ 중에서 $d(X,Y)$의 기대값을 가장 작게 추정한 값